In [63]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving EV_Cleaned.csv to EV_Cleaned (2).csv


In [64]:
df = pd.read_csv('EV_Cleaned.csv')
df.head()

,start_datetime,end_datetime,total_energy,station_name
0,8/25/2020 11:56,8/25/2020 12:08,1.477,600001
1,8/25/2020 12:20,8/25/2020 12:32,1.333,600002
2,8/25/2020 12:40,8/25/2020 13:01,2.330,600003
3,8/25/2020 13:05,8/25/2020 13:37,3.764,600005
4,8/25/2020 14:03,8/25/2020 14:23,2.297,600000


In [65]:
# Columns overview
print("Total columns:", df.shape[1])
print("Column names:")
for i, c in enumerate(df.columns, 1):
    print(f"{i:2d}. {c}")


# Data types + memory usage
print("\nData types:")
print(df.dtypes)

print("\nInfo (with memory usage):")
df.info(memory_usage="deep")

Total columns: 4
Column names:
 1. start_datetime
 2. end_datetime
 3. total_energy
 4. station_name

Data types:
start_datetime     object
end_datetime       object
total_energy      float64
station_name        int64
dtype: object

Info (with memory usage):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55379 entries, 0 to 55378
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   start_datetime  55379 non-null  object 
 1   end_datetime    55379 non-null  object 
 2   total_energy    55379 non-null  float64
 3   station_name    55379 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 7.6 MB


In [66]:
df['total_energy'] = pd.to_numeric(df['total_energy'], errors='coerce')

# Check again
print("Energy dtype:", df['total_energy'].dtypes)
df['total_energy'].head()


df['station_name'] = df['station_name'].astype(str)

print("Station dtype:", df['station_name'].dtypes)
df['station_name'].head()

Energy dtype: float64
Station dtype: object


,station_name
0,600001
1,600002
2,600003
3,600005
4,600000


In [67]:
import pandas as pd
import numpy as np

# --- 0) Work on a fresh copy
df = df.copy()

# --- 1) Dual-parse start_datetime (no in-place mutation of Index)
idx_str = df.index.astype(str)

idx1 = pd.to_datetime(idx_str, errors="coerce", dayfirst=True)
idx2 = pd.to_datetime(idx_str, errors="coerce", dayfirst=False)

# Combine: prefer idx1, fall back to idx2
idx_combined = (
    pd.Series(idx1, index=df.index)
      .combine_first(pd.Series(idx2, index=df.index))
)

bad_idx = int(idx_combined.isna().sum())
print(f"Invalid start_datetime after dual-parse: {bad_idx}")

# Keep only valid datetimes and set as index
df = df.loc[idx_combined.notna()].copy()
df.index = pd.DatetimeIndex(idx_combined.loc[idx_combined.notna()].values, name="start_datetime")

# --- 2) Dual-parse end_datetime the same way (if present)
if "end_datetime" in df.columns:
    s = df["end_datetime"].astype(str)
    e1 = pd.to_datetime(s, errors="coerce", dayfirst=True)
    e2 = pd.to_datetime(s, errors="coerce", dayfirst=False)
    df.loc[:, "end_datetime"] = (
        pd.Series(e1, index=df.index)
          .combine_first(pd.Series(e2, index=df.index))
    )

# --- 3) Ensure total_energy is numeric
df.loc[:, "total_energy"] = pd.to_numeric(df["total_energy"], errors="coerce")
bad_energy = int(df["total_energy"].isna().sum())
print(f"Non-numeric total_energy after coercion: {bad_energy}")

# Drop rows missing essentials and sort
df = df.dropna(subset=["total_energy"]).sort_index().copy()

print("Ready for resampling. Shape:", df.shape)
print(df.head())


/tmp/ipython-input-1729449809.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  idx1 = pd.to_datetime(idx_str, errors="coerce", dayfirst=True)
/tmp/ipython-input-1729449809.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  idx2 = pd.to_datetime(idx_str, errors="coerce", dayfirst=False)


Invalid start_datetime after dual-parse: 54794
Non-numeric total_energy after coercion: 0
Ready for resampling. Shape: (585, 4)
                 start_datetime         end_datetime  total_energy  \
start_datetime                                                       
1678-01-01      11/12/2021 7:58  2021-12-11 12:18:00        25.370   
1679-01-01      11/12/2021 8:11  2021-12-11 15:11:00         6.060   
1680-01-01      11/12/2021 8:15  2021-12-11 14:19:00        41.370   
1681-01-01      11/12/2021 8:18  2021-12-11 17:54:00        41.890   
1682-01-01      11/12/2021 8:34  2021-12-11 14:14:00        52.731   

               station_name  
start_datetime               
1678-01-01          1802533  
1679-01-01          1802661  
1680-01-01          1802531  
1681-01-01          1802650  
1682-01-01           600010  


In [68]:
# Hourly
hourly = df["total_energy"].resample("h").sum().asfreq("h", fill_value=0)
hourly_df = hourly.reset_index().rename(columns={"total_energy": "hourly_kWh"})

# Daily
daily = df["total_energy"].resample("d").sum().asfreq("d", fill_value=0)
daily_df = daily.reset_index().rename(columns={"total_energy": "daily_kWh"})


In [69]:
# Hourly
hourly = df["total_energy"].resample("H").sum().asfreq("H", fill_value=0)
hourly_df = hourly.reset_index().rename(columns={"total_energy": "hourly_kWh"})

# Daily
daily = df["total_energy"].resample("D").sum().asfreq("D", fill_value=0)
daily_df = daily.reset_index().rename(columns={"total_energy": "daily_kWh"})

/tmp/ipython-input-3805824729.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly = df["total_energy"].resample("H").sum().asfreq("H", fill_value=0)


In [70]:
# Hourly aggregation with missing timestamps filled as 0
hourly = (
    df["total_energy"]
    .resample("h")       # resample to every hour
    .sum()               # sum energy in that hour
    .asfreq("h", fill_value=0)  # fill missing timestamps with 0
)
hourly_df = hourly.reset_index().rename(columns={"total_energy": "hourly_kWh"})

# Daily aggregation with missing timestamps filled as 0
daily = (
    df["total_energy"]
    .resample("d")       # resample to every day
    .sum()
    .asfreq("d", fill_value=0)
)
daily_df = daily.reset_index().rename(columns={"total_energy": "daily_kWh"})

In [71]:
# Set thresholds
hourly_threshold = 1000   # kWh in one hour
daily_threshold = 5000    # kWh in one day

# Cap values in hourly
hourly_df["hourly_kWh"] = hourly_df["hourly_kWh"].clip(upper=hourly_threshold)

# Cap values in daily
daily_df["daily_kWh"] = daily_df["daily_kWh"].clip(upper=daily_threshold)

# Check if any values were capped
print("Max hourly after capping:", hourly_df["hourly_kWh"].max())
print("Max daily after capping:", daily_df["daily_kWh"].max())

# Show a few examples where data was capped
print("Hourly spikes capped:")
print(hourly_df[hourly_df["hourly_kWh"] >= hourly_threshold].head())

print("\nDaily spikes capped:")
print(daily_df[daily_df["daily_kWh"] >= daily_threshold].head())

Max hourly after capping: 73.177
Max daily after capping: 73.177
Hourly spikes capped:
Empty DataFrame
Columns: [start_datetime, hourly_kWh]
Index: []

Daily spikes capped:
Empty DataFrame
Columns: [start_datetime, daily_kWh]
Index: []


In [72]:
# --- For hourly data ---
hourly_df["hour"] = hourly_df["start_datetime"].dt.hour
hourly_df["day_of_week"] = hourly_df["start_datetime"].dt.dayofweek  # Monday=0, Sunday=6
hourly_df["is_weekend"] = hourly_df["day_of_week"].isin([5, 6]).astype(int)

# --- For daily data ---
daily_df["day_of_week"] = daily_df["start_datetime"].dt.dayofweek
daily_df["is_weekend"] = daily_df["day_of_week"].isin([5, 6]).astype(int)

In [73]:
# Ensure hourly is a DataFrame
hourly = df["total_energy"].resample("h").sum().reset_index()
hourly.columns = ["start_datetime", "hourly_kWh"]

# Now create lag features
hourly["lag_1h"] = hourly["hourly_kWh"].shift(1)     # demand 1 hour ago
hourly["lag_24h"] = hourly["hourly_kWh"].shift(24)   # demand same hour yesterday
hourly["lag_168h"] = hourly["hourly_kWh"].shift(168) # demand same hour last week

# Drop rows with NaN from shifting
hourly = hourly.dropna().reset_index(drop=True)

print(hourly.head(10))

daily_df = (
    df["total_energy"]
    .resample("d").sum()          # lowercase 'd'
    .asfreq("d", fill_value=0)
    .reset_index()
    .rename(columns={"total_energy": "daily_kWh"})
)


       start_datetime  hourly_kWh  lag_1h  lag_24h  lag_168h
0 1678-01-08 00:00:00         0.0     0.0      0.0     25.37
1 1678-01-08 01:00:00         0.0     0.0      0.0      0.00
2 1678-01-08 02:00:00         0.0     0.0      0.0      0.00
3 1678-01-08 03:00:00         0.0     0.0      0.0      0.00
4 1678-01-08 04:00:00         0.0     0.0      0.0      0.00
5 1678-01-08 05:00:00         0.0     0.0      0.0      0.00
6 1678-01-08 06:00:00         0.0     0.0      0.0      0.00
7 1678-01-08 07:00:00         0.0     0.0      0.0      0.00
8 1678-01-08 08:00:00         0.0     0.0      0.0      0.00
9 1678-01-08 09:00:00         0.0     0.0      0.0      0.00


In [74]:
# Hourly lag features
hourly["lag_1h"] = hourly["hourly_kWh"].shift(1)     # demand 1 hour ago
hourly["lag_24h"] = hourly["hourly_kWh"].shift(24)   # demand same hour yesterday
hourly["lag_168h"] = hourly["hourly_kWh"].shift(168) # demand same hour last week
# Daily lag features
daily_df["lag_1d"]  = daily_df["daily_kWh"].shift(1)
daily_df["lag_7d"]  = daily_df["daily_kWh"].shift(7)
daily_df["lag_30d"] = daily_df["daily_kWh"].shift(30)

hourly_feat = hourly_df.dropna().reset_index(drop=True)
daily_feat  = daily_df.dropna().reset_index(drop=True)

print(hourly_feat.head(3))
print(daily_feat.head(3))


       start_datetime  hourly_kWh  hour  day_of_week  is_weekend
0 1678-01-01 00:00:00       25.37     0            5           1
1 1678-01-01 01:00:00        0.00     1            5           1
2 1678-01-01 02:00:00        0.00     2            5           1
  start_datetime  daily_kWh  lag_1d  lag_7d  lag_30d
0     1678-01-31        0.0     0.0     0.0    25.37
1     1678-02-01        0.0     0.0     0.0     0.00
2     1678-02-02        0.0     0.0     0.0     0.00


In [75]:
# 7-day & 30-day trailing means
daily_df = daily_df.sort_values("start_datetime").reset_index(drop=True)

daily_df["ma_7d"]  = daily_df["daily_kWh"].rolling(7,  min_periods=1).mean()
daily_df["ma_30d"] = daily_df["daily_kWh"].rolling(30, min_periods=1).mean()

daily_df.head(1)

,start_datetime,daily_kWh,lag_1d,lag_7d,lag_30d,ma_7d,ma_30d
0,1678-01-01,25.37,NaN,NaN,NaN,25.37,25.37


In [76]:
# 24-hour trailing mean (same hour past day) and weekly (past 168 hours)
hourly_df = hourly_df.sort_values("start_datetime").reset_index(drop=True)

hourly_df["ma_24h"]  = hourly_df["hourly_kWh"].rolling(24,   min_periods=1).mean()
hourly_df["ma_168h"] = hourly_df["hourly_kWh"].rolling(168,  min_periods=1).mean()  # 7 * 24

hourly_df.head(1)

,start_datetime,hourly_kWh,hour,day_of_week,is_weekend,ma_24h,ma_168h
0,1678-01-01,25.37,0,5,1,25.37,25.37


In [62]:
# Save hourly data
hourly_df.to_csv("hourly_clean.csv", index=False)

# Save daily data
daily_df.to_csv("daily_clean.csv", index=False)

print("✅ Cleaned files saved: hourly_clean.csv & daily_clean.csv")

✅ Cleaned files saved: hourly_clean.csv & daily_clean.csv
